# MSA 2023 Phase 2 - Part 3

I followed the original/previous exemplar but struggled with creating the virtual environment/Docker container.
I have found more success but have pretty much used all of the original code from the exemplar.
However, I have used a regressor instead of a classifier to add in some originality.

In [1]:
# Azure Machine Learning SDK core
from azureml.core import Workspace
from azureml.core.model import Model

# Scikit-learn and others
from sklearn import datasets; iris = datasets.load_iris()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

#for later
import pandas as pd
import numpy as np


### Initialising the workspace for model deployment

In [3]:
ws = Workspace.from_config(path="config.json")

### Registering model onto Azure

In [5]:
model = Model.register(ws, model_name="lr_model", model_path="lr_model.pkl")

Registering model lr_model


### Entry Script

My entry script has been altered to accomdodate a regrssor instead of a classifier. I assumed that the input data as a csv, and had to make a dataframe instead of a numpy array. 

### Test and share endpoint for marking

To introduce a new dataset, the input data needs to follow these steps:
1. Add in 'import pandas as pd' to import the pandas library.
2. The data must be introduced into a dataframe: df = pd.read_csv(raw_dataset)
3. The dataset must be converted into a dictionary object to be JSON serializable. Use the Dataframe.to_dict() function:  dict_df = df.to_dict()
4. The dictionary object created must be an argument for json.dumps():     str.encode(json.dumps(dictionary_object))

import pandas as pd
df = pd.read_csv('changed_exam_scores.csv',index_col=0)
data = df.to_dict()                                

# wlgoi
    data = json.loads(request)
    data = np.array(data["data"])
    result = model.predict(data)
    return result

df = pd.read_csv('changed_exam_scores.csv',index_col=0)
npa = np.array(df)
d1 = npa.tolist() 
data = {'data': d1}
body = str.encode(json.dumps(data))

In [28]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
df = pd.read_csv('changed_exam_scores.csv',index_col=0)
npa = np.array(df)
d1 = npa.tolist() 
data = {'data': d1}
body = str.encode(json.dumps(data))

url = 'https://msa2023-phase2-azure-xswgk.australiaeast.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = ''
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'lr-model-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 424
server: azureml-frontdoor
date: Wed, 09 Aug 2023 09:56:52 GMT
content-type: application/json
content-length: 83
x-ms-run-function-failed: False
x-ms-server-version: azmlinfsrv/0.8.4
x-ms-request-id: 1d53da34-b71a-44ea-bc16-922441e8c6f8
x-request-id: 1d53da34-b71a-44ea-bc16-922441e8c6f8
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: lr-model-1
connection: close


{"message": "An unexpected internal error occurred. Check the logs for more info."}


### 8. Cleanup

Please do not delete or turn off any resources until the MSA team has completed marking. Once finished marking, this will be announced on Discord, after which (to avoid incurring unnecessary costs):
1. Delete your models and endpoints from [Machine Learning Studio](https://ml.azure.com/).
1. Delete or turn off any resources you created, as explained in [this section](https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-azure-ml-in-a-day?view=azureml-api-2#delete-all-resources) of the Azure Machine Learning documentation.